# GOLDILOX Insights - Shared Views Setup

This notebook creates a **data share** back to the Goldilox provider for troubleshooting and support analysis. Only run this notebook when requested by the Goldilox team.

**Run this notebook as ACCOUNTADMIN** (or a role with `CREATE DATABASE`, `CREATE SHARE` privileges).

### What this notebook does

1. Creates a `GOLDILOX_INSIGHTS_SHARE` database with copies of app data
2. Sets up a Snowflake data share to the Goldilox provider account
3. Creates a daily scheduled task to keep the shared data current

### What data is shared

| Table | Description | Data Range |
|---|---|---|
| `QUERY_HISTORY` | Query execution history with query profiles | Last 1 month (daily refresh) |
| `RECOMMENDATION_RESULT` | Generated clustering key recommendations | All |
| `TRAINING_DATA` | Partition pruning training data used by the ML model | All |

> No raw customer data (table contents) is shared — only query metadata and app-generated analysis results.

### Prerequisites

- The GOLDILOX Insights app is installed and has run at least once
- You have been asked by the Goldilox team to set up data sharing

> **App name:** This notebook assumes the default app name `GOLDILOX_INSIGHTS`. If you renamed the app during installation, update the `APP_NAME` variable in the Configuration cell below.

---

## Configuration

The default app name is `GOLDILOX_INSIGHTS`. If you renamed the app during installation, update the value below. Otherwise, run this cell as-is.

In [ ]:
-- Default app name (update only if you renamed the app during installation)
SET APP_NAME = 'GOLDILOX_INSIGHTS';
SET QH_VIEW = $APP_NAME || '.SHARED_VIEWS.QUERY_HISTORY';
SET RR_VIEW = $APP_NAME || '.SHARED_VIEWS.RECOMMENDATION_RESULT';
SET TD_VIEW = $APP_NAME || '.SHARED_VIEWS.TRAINING_DATA';
SELECT $APP_NAME AS APP_NAME;

---

## Step 1: Create share database and copy data

This step creates a dedicated database for the share and copies app data into it.

In [ ]:
-- Create the database and schema
CREATE DATABASE IF NOT EXISTS GOLDILOX_INSIGHTS_SHARE;
CREATE SCHEMA   IF NOT EXISTS GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES;

-- Copy table data from the app's shared views
CREATE OR REPLACE TABLE GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.QUERY_HISTORY AS
    SELECT * FROM IDENTIFIER($QH_VIEW) WHERE START_TIME > DATEADD('MONTH', -1, CURRENT_DATE());

CREATE OR REPLACE TABLE GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.RECOMMENDATION_RESULT AS
    SELECT * FROM IDENTIFIER($RR_VIEW);

CREATE OR REPLACE TABLE GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.TRAINING_DATA AS
    SELECT * FROM IDENTIFIER($TD_VIEW);

### Create the data share

This creates a Snowflake share and grants the Goldilox provider account access to the shared tables.

In [ ]:
-- Create the share and grant access
CREATE SHARE IF NOT EXISTS GOLDILOX_INSIGHTS_SHARE;

GRANT USAGE ON DATABASE GOLDILOX_INSIGHTS_SHARE TO SHARE GOLDILOX_INSIGHTS_SHARE;
GRANT USAGE ON SCHEMA   GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES TO SHARE GOLDILOX_INSIGHTS_SHARE;
GRANT SELECT ON ALL TABLES IN SCHEMA GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES TO SHARE GOLDILOX_INSIGHTS_SHARE;

-- Add the Goldilox provider account to the share
-- This is the Goldilox Technology provider account that will receive the shared data
ALTER SHARE GOLDILOX_INSIGHTS_SHARE ADD ACCOUNTS = SEFURGX.ALB16562;

---

## Step 2: Create a daily refresh task

This task rebuilds the shared tables daily so the Goldilox team always has current data for analysis.

> **Important:**
> - Update the **warehouse name** (`GOLDILOX_APP_TASK_WAREHOUSE`) to a warehouse available in your environment.
> - If you renamed the app during installation, update the three references to `GOLDILOX_INSIGHTS` in the task body below. Session variables do not persist inside task bodies, so these must be hardcoded.

In [ ]:
-- >>> Update GOLDILOX_APP_TASK_WAREHOUSE to a warehouse available in your environment <<<

CREATE SCHEMA IF NOT EXISTS GOLDILOX_INSIGHTS_SHARE.TASKS;
CREATE OR REPLACE TASK GOLDILOX_INSIGHTS_SHARE.TASKS.REFRESH_DATA
  WAREHOUSE = GOLDILOX_APP_TASK_WAREHOUSE
  SCHEDULE = 'USING CRON 30 06 * * * UTC'
  COMMENT = 'Rebuild shared tables daily'
AS
BEGIN
  CREATE OR REPLACE TABLE GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.QUERY_HISTORY
  COPY GRANTS
  AS
  SELECT *
  FROM GOLDILOX_INSIGHTS.SHARED_VIEWS.QUERY_HISTORY
  WHERE START_TIME > DATEADD('MONTH', -1, CURRENT_DATE());

  CREATE OR REPLACE TABLE GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.RECOMMENDATION_RESULT
  COPY GRANTS
  AS
  SELECT *
  FROM GOLDILOX_INSIGHTS.SHARED_VIEWS.RECOMMENDATION_RESULT;

  CREATE OR REPLACE TABLE GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.TRAINING_DATA
  COPY GRANTS
  AS
  SELECT *
  FROM GOLDILOX_INSIGHTS.SHARED_VIEWS.TRAINING_DATA;

  GRANT SELECT ON ALL TABLES IN SCHEMA GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES TO SHARE GOLDILOX_INSIGHTS_SHARE;
END;

-- Enable and run the task immediately
ALTER TASK GOLDILOX_INSIGHTS_SHARE.TASKS.REFRESH_DATA RESUME;
EXECUTE TASK GOLDILOX_INSIGHTS_SHARE.TASKS.REFRESH_DATA;

---

## Step 3: Verify setup

Run the cells below to confirm the share and task are set up correctly.

In [ ]:
-- Verify shared tables exist and have data
SELECT 'QUERY_HISTORY' AS TABLE_NAME, COUNT(*) AS ROW_COUNT FROM GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.QUERY_HISTORY
UNION ALL
SELECT 'RECOMMENDATION_RESULT', COUNT(*) FROM GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.RECOMMENDATION_RESULT
UNION ALL
SELECT 'TRAINING_DATA', COUNT(*) FROM GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.TRAINING_DATA;

In [ ]:
-- Verify the share is configured
SHOW SHARES LIKE 'GOLDILOX_INSIGHTS_SHARE';

In [ ]:
-- Verify the refresh task is running
SHOW TASKS IN SCHEMA GOLDILOX_INSIGHTS_SHARE.TASKS;

---

## Tear Down

Run the cells below to remove all objects created by this notebook. This will stop data sharing with the Goldilox provider.

> **Warning:** This will immediately stop the data share and delete all shared data. Please consult the Goldilox team at **support@goldilox.com** before proceeding.

To remove all objects created by this notebook, run the following SQL in a worksheet:

```sql
-- Stop the refresh task
ALTER TASK IF EXISTS GOLDILOX_INSIGHTS_SHARE.TASKS.REFRESH_DATA SUSPEND;

-- Remove the data share
DROP SHARE IF EXISTS GOLDILOX_INSIGHTS_SHARE;

-- Drop the database (removes all shared tables, schemas, and the task)
DROP DATABASE IF EXISTS GOLDILOX_INSIGHTS_SHARE;
```